In [1]:
import sys
sys.path.append('/home/gvolta/XENONnT/LedAnalysis/Script')
from PMTanalysis import *
PMT = PMTanalyzer('/home/gvolta/XENONnT/LedAnalysis/Script')

/home/gvolta/XENONnT/LedAnalysis/Script/configuration.ini


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle
import matplotlib.colors as mcolors
#mpl.use('Agg')

In [3]:
import strax
import straxen
strax.Mailbox.DEFAULT_MAX_MESSAGES = 2
st = straxen.contexts.strax_SPE()

/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
Config = cp.ConfigParser()
Config.read(PMT.config_file)

['/home/gvolta/XENONnT/LedAnalysis/Script/configuration.ini']

## _Test of PMT analyzer class_

In [5]:
runs = st.select_runs(run_mode='LED*')

Checking data availability: 100%|██████████| 2/2 [00:24<00:00, 10.10s/it]


In [6]:
run_led = str(PMT.run_led)
run_noise = str(PMT.run_noise)

In [ ]:
data_rr = st.get_array(run_led, 'raw_records', max_messages=3, max_workers=5, seconds_range=(0,20))

In [ ]:
amplitude = PMT.get_amplitude(data=data_rr)

In [ ]:
df_fit, df_led, window = PMT.get_ledwindow(amplitude=amplitude, channels=[i for i in range(0,249)])

In [9]:
led_window   = [Config.getint('window_pars','led_windows_left'), Config.getint('window_pars','led_windows_right')]
noise_window = [Config.getint('window_pars','noise_windows_left'), Config.getint('window_pars','noise_windows_right')]

st = st.new_context(config=dict(led_window=(led_window[0],led_window[1]), 
                                noise_window=(noise_window[0], noise_window[1])))
st.show_config('led_calibration')

,option,default,current,applies_to,help
0,led_window,"(150, 275)","(143, 168)","(led_calibration,)",Window (samples) where we expect the signal in LED calibration
1,noise_window,"(0, 125)","(5, 30)","(led_calibration,)",Window (samples) to analysis the noise
2,channel_list,"(0, 248)",<OMITTED>,"(led_calibration,)","Three different light level for XENON1T: (0,36), (37,126), (127,248). Defalt value: all the PMTs"
3,pax_raw_dir,/data/xenon/raw,<OMITTED>,"(raw_records,)",Directory with raw pax datasets
4,stop_after_zips,0,<OMITTED>,"(raw_records,)",Convert only this many zip files. 0 = all.
5,events_per_chunk,50,<OMITTED>,"(raw_records,)",Number of events to yield per chunk
6,samples_per_record,110,<OMITTED>,"(raw_records,)",Number of samples per record


In [10]:
data_led = st.get_array(run_led, 'led_calibration', max_workers=10)

In [11]:
data_led

array([(  6,   5, 4,   0, 1519037400000123910, 10, 600),
       ( 22,   3, 7,   1, 1519037400000123910, 10, 600),
       (  9,   3, 4,   2, 1519037400000123910, 10, 600), ...,
       (253,  95, 1, 227, 1519037880900775250, 10, 600),
       (272, 121, 3, 235, 1519037880900775250, 10, 600),
       ( 21,   4, 2, 243, 1519037880900775250, 10, 600)],
      dtype=[(('Area averaged in integration windows', 'area'), '<i4'), (('Amplitude in LED window', 'amplitude_led'), '<i4'), (('Amplitude in off LED window', 'amplitude_noise'), '<i4'), (('Channel', 'channel'), '<i2'), (('Start time of the interval (ns since unix epoch)', 'time'), '<i8'), (('Time resolution in ns', 'dt'), '<i2'), (('Length of the interval in samples', 'length'), '<i4')])

In [ ]:
def ScalingSpectrum(data, n_channel_s = np.arange(0, 249, 1)):
    
    # In order to subtract out the contribution of the noise to the amplitude spectrum, we will assume that 
    # the fraction of SPE signals with amplitude below a threshold of(3-7) ADC counts is very small. 
    # We then scale down the off-time amplitude spectrum such that the total counts below the 
    # (3-7) ADC count threshold is the same as in the LED spectrum.
    # The spectrum also contains contributions of 2 or more photoelectrons. From the scaling down factor 
    # of the noise s, assuming a Poisson distribution of photoelectrons we estimate that the average 
    # number of photoelectrons (occupancy) in the LED run was lambda = -ln(s) = 0.566.
    # The fraction of events with 2 or more photoelectrons is then 1- exp(-lambda)(1+lambda) = 0.111. 
    # The contribution of 2 or more photoelectrons leads to a slight over-estimate in the acceptances calculated.
    
    df_spe = pd.DataFrame({'channel': [ ], 
                           'spectrum led': [ ], 'bins led': [ ],
                           'spectrum noise': [ ], 'bins noise': [ ],
                           'spectrum noise scaled 3 bin': [ ], 'occupancy 3 bin': [ ],
                           'spectrum noise scaled 4 bin': [ ], 'occupancy 4 bin': [ ],
                           'spectrum noise scaled 5 bin': [ ], 'occupancy 5 bin': [ ],
                           'spectrum noise scaled 6 bin': [ ], 'occupancy 6 bin': [ ],
                           'spectrum noise scaled 7 bin': [ ], 'occupancy 7 bin': [ ]})
    
    #SPE = np.zeros((len(n_channel_s)), dtype = datatype)

    for n_channel in tqdm(n_channel_s):
        arr = data[data['channel'] == n_channel]

        LED, bins_LED = np.histogram(arr['amplitudeLED'], bins=250, range=(0,500))
        bins_LED_center = 0.5 * (bins_LED[1:] + bins_LED[:-1])
        noise, bins_noise = np.histogram(arr['amplitudeNOISE'], bins=250, range=(0,500))
        bins_noise_center = 0.5 * (bins_noise[1:] + bins_noise[:-1])
        
        SPE[n_channel]['pmt'] = n_channel
        SPE[n_channel]['spectrumLED'] = LED
        SPE[n_channel]['bins_LED_center'] = bins_LED_center
        SPE[n_channel]['spectrumNOISE'] = noise
        SPE[n_channel]['bins_NOISE_center'] = bins_noise_center
        
        for i in range(3,8):
            ADC_correction = i
            scaling_coeff = LED[:i].sum()/noise[:i].sum() 
            SPE[n_channel]['spectrumNOISE_scaled_'+str(i)+'bin'] = noise*scaling_coeff
            SPE[n_channel]['occupancy_'+str(i)+'bin'] = -np.log(scaling_coeff)
    
    return SPE